<h1>Car Price Prediction</h1>


In [1]:
import numpy as np
import pandas as pd
import json
import re
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.warn("this will not show")

In [3]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_info_columns', 500)
pd.set_option('display.max_info_rows', 2000)
pd.set_option('display.expand_frame_repr', True)
pd.set_option('display.width', 2000)

In [4]:
def show_nans(df, limit):
    missing = df.isnull().mean()*100
    return missing.loc[lambda x : x > limit] 

In [5]:
def n_sil(x):
    for i in range(0, len(x)):
        if isinstance(x[i], list):
            for a in x[i]:
                if a == "\n":
                    x[i].remove(a)
                elif "\n" in a:
                    x[i].remove(a)
                    b = a.replace("\n", "")
                    x[i].append(b)
        elif isinstance(x[i], str):
            x[i] = x[i].replace("\n", "")

In [6]:
def Nan_to_list(data, column_name):
    NaN_rows = data[column_name].isnull()
    # Change the type of NaNs from 'float' to (empty) 'list'
    data.loc[NaN_rows, column_name] = data.loc[NaN_rows, column_name].apply(lambda x: [])

In [7]:
def unique_values(data, column_name):
    unique_vals = set()
    for rows in data[column_name]:
        unique_vals.update(rows)
    return list(unique_vals)

In [8]:
def fill_most(oto, group_col, col_name):
    for group in list(oto[group_col].unique()):
        try:
            grp_inx = list(oto[oto[group_col] == group][col_name].index)
            oto[col_name].iloc[grp_inx] = oto[col_name].iloc[grp_inx].fillna(oto[oto[group_col]==group][col_name].mode()[0])
        except Exception as e:
            print("{} 'da {} Error Oldu".format(group, e))
            print("devam ediliyor")

In [9]:
def liststring(df, col):             #Listeleri stringlere dönüştürür.
    df[col] = [','.join(l) if type(l) == list else l for l in df[col]]
    
def emptytonan(df, col):             #Stringlerdeki boşlukları NaN'a dönüştürür.
    df[col] = [np.NaN if len(l)==0 else l for l in df[col]]

In [10]:
veri = []
for line in open("scout_car.json", "r"):
    veri.append(json.loads(line))
    
#pd.read_json("scout_car.json", lines = True)

In [11]:
df = pd.DataFrame(veri)

In [12]:
df.shape

(15919, 54)

In [13]:
df.rename(columns={"\nComfort & Convenience\n":"Comfort & Convenience", "\nEntertainment & Media\n":"Entertainment & Media", "\nExtras\n":"Extras", "\nSafety & Security\n":"Safety & Security"}, inplace=True)

In [14]:
#Verilerin %60ından fazlası null olan sütunlar düşürüldü.
df.drop(["null", 
         "kW", 
         "Electricity consumption", 
         "Last Service Date", 
         "Other Fuel Types", 
         "Availability", 
         "Last Timing Belt Service Date", 
         "Available from", 
         "Inspection new", 
         "Emission Label", 
         "Model Code"], axis=1, inplace=True)

#Tekrarlı veya gereksiz sütunlar düşürüldü.Body=body_type idi.
df.drop(["Body", "Offer Number", "prev_owner", "Non-smoking Vehicle", 
         "Weight", "url", "Full Service", "First Registration",
        "Body Color Original", "description", "short_description"], axis=1, inplace=True)

In [15]:
df.shape

(15919, 32)

In [16]:
for i in df.columns:
    Nan_to_list(df, i)

In [17]:
yeni_list = []
for i in df.columns[8:]:
    yeni_list.append(i)

In [18]:
yeni_list

['Previous Owners',
 'Next Inspection',
 'Warranty',
 'Make',
 'Model',
 'Body Color',
 'Paint Type',
 'Upholstery',
 'Nr. of Doors',
 'Nr. of Seats',
 'Gearing Type',
 'Displacement',
 'Cylinders',
 'Drive chain',
 'Fuel',
 'Consumption',
 'CO2 Emission',
 'Emission Class',
 'Comfort & Convenience',
 'Entertainment & Media',
 'Extras',
 'Safety & Security',
 'Gears',
 'Country version']

In [19]:
for i in yeni_list:
    n_sil(df[i])

In [20]:
for i in yeni_list:
    n_sil(df[i])

In [21]:
df_ilk = df.copy()

# Previous Owner

In [22]:
def prev_own(df, clm):   #Prev Owner içinde sadece "0", "1", "2", "3", "4"(prev_owner'da sadece bu değerler olduğu için) olanları alır. 
    for i in range(0, len(df[clm])):
        if type(df[clm][i]) == list:
            for a in df[clm][i]:
                if a in ["0", "1", "2", "3", "4"]:
                    df[clm][i] = a

In [23]:
prev_own(df, "Previous Owners")

In [24]:
df["Previous Owners"].value_counts(dropna=False)

1     8294
[]    6640
2      778
0      188
3       17
4        2
Name: Previous Owners, dtype: int64

In [25]:
df["Previous Owners"][df.Type=="New"] = df["Previous Owners"][df.Type=="New"].fillna("0")

In [26]:
emptytonan(df, "Previous Owners")

In [27]:
fill_most(df, "registration", "Previous Owners")

09/2019 'da index out of bounds Error Oldu
devam ediliyor
11/2019 'da index out of bounds Error Oldu
devam ediliyor
12/2019 'da index out of bounds Error Oldu
devam ediliyor


In [28]:
df["Previous Owners"] = df["Previous Owners"].fillna("1")

# Warranty

In [29]:
def warranty(df, clm):   #Warranty sütunundaki alakasız bilgileri siler.
    for i in range(0, len(df[clm])):
        if type(df[clm][i]) == list:
            df[clm][i] = [x for x in df[clm][i] if x.isnumeric() or "month" in x]
        elif type(df[clm][i]) == str:
            b = []
            b.append(df[clm][i])
            df[clm][i] = [x for x in b if x.isnumeric() or "month" in x]

In [30]:
warranty(df, "Warranty")

# Liste Boş Elemanları Silme

In [31]:
def boslistsil(df, clm):       # Sütundaki listelerin içindeki boşlukları siler.
    for i in range(0, len(df[clm])):
        if type(df[clm][i]) == list:
            df[clm][i] = [x for x in df[clm][i] if len(x)!=0]

In [32]:
for i in ["CO2 Emission",
         "Emission Class",
         "Type"]:
    boslistsil(df, i)

# Listeler Stringe

In [33]:
for i in ["Warranty", 
          "Model", 
          "Body Color",  
          "Upholstery", 
          "Nr. of Doors",
          "Nr. of Seats",
          "Gearing Type",
          "Displacement",
          "Cylinders",
          "Drive chain",
          "CO2 Emission",
          "Emission Class",
          "Paint Type", 
          "Country version", 
          "Gears",
          "Next Inspection"]:
    try:
        liststring(df, i)
        emptytonan(df, i)
    except:
        print("{} 'da {} hatası çıktı".format(i, Exception))

# Fuel

In [34]:
liststring(df, "Fuel")

In [35]:
df.Fuel = df.Fuel.str.split("/")

In [36]:
df.Fuel.value_counts()

[Diesel (Particulate Filter)]                                                                                         4315
[Super 95]                                                                                                            3338
[Gasoline]                                                                                                            3175
[Diesel]                                                                                                              2984
[Super 95 ,  Regular, Benzine 91]                                                                                      424
                                                                                                                      ... 
[Super Plus 98 ,  Super E10 95]                                                                                          1
[Regular, Benzine 91 ,  Super 95 ,  Regular, Benzine E10 91 ,  Super E10 95 ,  Super Plus 98 ,  Super Plus E10 98]       1
[Super 95 ,  Reg

In [37]:
for i in range(0, len(df.Fuel)):
    df.Fuel[i] = [b.strip() for b in df.Fuel[i]]

In [38]:
liststring(df, "Fuel")

# Type

In [39]:
liststring(df, "Type")

In [40]:
df.Type = df.Type.str.split("/")

In [41]:
for i in range(0, len(df.Type)):
    df.Type[i] = [b.strip() for b in df.Type[i]]

In [42]:
liststring(df, "Type")

In [43]:
def conversion(df, clm, wtc=[]):
    for i in wtc:
        df[clm][df[clm].str.contains(i, regex=False)] = i

In [44]:
conversion(df, "Type", wtc=["Used", "Pre-registered", "New", "Employee's car", "Demonstration"])

In [45]:
df.Type[df.Type == ""] = "Used"

# Warranty

In [46]:
fill_most(df, "make_model", "Warranty") #Warranty NANları silindi.

Audi A2 'da index out of bounds Error Oldu
devam ediliyor


In [47]:
df.Warranty[df.Warranty.isnull()] = "12 months"

# body_type

In [48]:
emptytonan(df, "body_type")

In [49]:
fill_most(df, "make_model", "body_type")

# vat

In [50]:
liststring(df, "vat")

In [51]:
emptytonan(df, "vat")

In [52]:
fill_most(df, "make_model", "vat")

# km

In [53]:
df.km = df.km.str.strip(" km")

In [54]:
df.km = df.km.str.replace(",", "")

In [55]:
df.km[df.km.str[0]== "-"] = np.NaN

In [56]:
fill_most(df, "make_model", "km")

# registration

In [57]:
df.registration[df.registration.str[0]== "-"] = np.NaN

In [58]:
fill_most(df, "km", "registration")

0 'da index out of bounds Error Oldu
devam ediliyor
281 'da index out of bounds Error Oldu
devam ediliyor
196 'da index out of bounds Error Oldu
devam ediliyor
141 'da index out of bounds Error Oldu
devam ediliyor
68485 'da index out of bounds Error Oldu
devam ediliyor
81800 'da index out of bounds Error Oldu
devam ediliyor
142 'da index out of bounds Error Oldu
devam ediliyor
39962 'da index out of bounds Error Oldu
devam ediliyor
32084 'da index out of bounds Error Oldu
devam ediliyor
20768 'da index out of bounds Error Oldu
devam ediliyor
34164 'da index out of bounds Error Oldu
devam ediliyor
325 'da index out of bounds Error Oldu
devam ediliyor
89982 'da index out of bounds Error Oldu
devam ediliyor
77 'da index out of bounds Error Oldu
devam ediliyor
89692 'da index out of bounds Error Oldu
devam ediliyor
784 'da index out of bounds Error Oldu
devam ediliyor
497 'da index out of bounds Error Oldu
devam ediliyor


In [59]:
df.registration.iloc[df[df.km=="0"].index] = df.registration.iloc[df[df.km=="0"].index].fillna("01/2020")

In [60]:
fill_most(df, "make_model", "registration")

# hp

In [61]:
df.hp = df.hp.str.replace(" kW", "")

In [62]:
df.hp[df.hp.str[0]== "-"] = np.NaN

In [63]:
fill_most(df, "make_model", "hp")

# Previous Owner

In [64]:
fill_most(df, "registration", "Previous Owners")

In [65]:
df["Previous Owners"] = df["Previous Owners"].fillna("1")

# Body Color & Paint Type

In [66]:
fill_most(df, "make_model", "Body Color")

In [67]:
fill_most(df, "Body Color", "Paint Type")

Gold 'da index out of bounds Error Oldu
devam ediliyor


In [68]:
df["Paint Type"] = df["Paint Type"].fillna("Perl effect")

# Upholstery

In [69]:
fill_most(df, "make_model", "Upholstery")

In [70]:
df.Upholstery[df.Upholstery=="Other, Other"] = "Other"

# Nr. of Doors & Nr. of Seats

In [71]:
fill_most(df, "make_model", "Nr. of Doors")

In [72]:
fill_most(df, "make_model", "Nr. of Seats")

# Displacement

In [73]:
df.Displacement = df.Displacement.str.replace(" cc", "")

In [74]:
df.Displacement = df.Displacement.str.replace(",", "")

In [75]:
fill_most(df, "make_model", "Displacement")

# Cylinders

In [76]:
fill_most(df, "make_model", "Cylinders")

Audi A2 'da index out of bounds Error Oldu
devam ediliyor


In [77]:
df["Cylinders"] = df["Cylinders"].fillna("3")

# Drive chain

In [78]:
fill_most(df, "make_model", "Drive chain")

# Fuel

In [79]:
conversion(df, "Fuel", wtc = ["Electric,Gasoline", "Electric", "Diesel", "Benzine", "Gasoline", "gas", "Super", "CNG", "LPG", "Other"])

In [80]:
df.Fuel[df.Fuel=="Super"] = "Benzine"

In [81]:
df.Fuel[df.Fuel=="gas"] = "Gasoline"

In [82]:
df.Fuel[df.Fuel=="CNG"] = "Gasoline"

In [83]:
df.Fuel[df.Fuel=="LPG"] = "Gasoline"

In [84]:
df.Fuel[df.Fuel=="Other"] = "Diesel"

In [85]:
df["Fuel"].value_counts(dropna=False)

Diesel      7305
Benzine     5293
Gasoline    3316
Electric       5
Name: Fuel, dtype: int64

# CO2

In [86]:
df["CO2 Emission"] = df["CO2 Emission"].str.replace(' g CO2/km ', '')

In [87]:
df["CO2 Emission"] = df["CO2 Emission"].str.strip('(comb)')

In [88]:
df["CO2 Emission"] = df["CO2 Emission"].str.replace(',', '')

In [89]:
df["CO2 Emission"][(df["CO2 Emission"]=="Benzine")|(df["CO2 Emission"]=="Gasoline")|(df["CO2 Emission"]=="Diesel")] = np.NaN

In [90]:
fill_most(df, "Fuel", "CO2 Emission")

# Emission Class

In [91]:
df["Emission Class"][(df["Emission Class"]=="Benzine")|(df["Emission Class"]=="Gasoline")|(df["Emission Class"]=="Diesel")] = np.NaN

In [92]:
df["Emission Class"][(df["Emission Class"]=="Euro 6d")] = "Euro 6d-TEMP"

In [93]:
fill_most(df, "CO2 Emission", "Emission Class")

1060 'da index out of bounds Error Oldu
devam ediliyor
990 'da index out of bounds Error Oldu
devam ediliyor
5 'da index out of bounds Error Oldu
devam ediliyor
80 'da index out of bounds Error Oldu
devam ediliyor
14 'da index out of bounds Error Oldu
devam ediliyor
13983 'da index out of bounds Error Oldu
devam ediliyor
1 'da index out of bounds Error Oldu
devam ediliyor


In [94]:
df["Emission Class"] = df["Emission Class"].fillna("Euro 6")

# Gears

In [95]:
df["Gears"][(df["Gears"]=="Benzine")|(df["Gears"]=="Gasoline")|(df["Gears"]=="Diesel")] = np.NaN

In [96]:
fill_most(df, "make_model", "Gears")

Audi A2 'da index out of bounds Error Oldu
devam ediliyor


In [97]:
df["Gears"] = df["Gears"].fillna("6")

# Country version

In [98]:
fill_most(df, "make_model", "Country version")

# Consumption

In [99]:
df["Consumption"] = df["Consumption"].str[0].str[0].str.extract("(\d.\d|\d)")[0]

In [100]:
fill_most(df, "make_model", "Consumption")

# Next Inspection

In [101]:
df["Next Inspection"] = df["Next Inspection"].str.extract("(\d\d/\d\d\d\d)")[0]

In [102]:
df["Next Inspection"] = pd.to_datetime(df["Next Inspection"], format="%m/%Y")

In [103]:
fill_most(df, "registration", "Next Inspection")

01/2020 'da index 0 is out of bounds for axis 0 with size 0 Error Oldu
devam ediliyor
07/2019 'da index 0 is out of bounds for axis 0 with size 0 Error Oldu
devam ediliyor
08/2019 'da index 0 is out of bounds for axis 0 with size 0 Error Oldu
devam ediliyor
11/2019 'da index 0 is out of bounds for axis 0 with size 0 Error Oldu
devam ediliyor


In [104]:
fill_most(df, "make_model", "Next Inspection")

# Comfort & Convenience

In [105]:
liststring(df, "Comfort & Convenience")

In [106]:
emptytonan(df, "Comfort & Convenience")

In [107]:
fill_most(df, "make_model", "Comfort & Convenience")

In [108]:
df = df.join(df["Comfort & Convenience"].str.get_dummies(sep=",").add_prefix("cc_"))

# Entertainment & Media

In [109]:
liststring(df, "Entertainment & Media")

In [110]:
emptytonan(df, "Entertainment & Media")

In [111]:
fill_most(df, "make_model", "Entertainment & Media")

In [112]:
df = df.join(df["Entertainment & Media"].str.get_dummies(sep=",").add_prefix("em_"))

# Extras

In [113]:
liststring(df, "Extras")

In [114]:
emptytonan(df, "Extras")

In [115]:
fill_most(df, "make_model", "Extras")

In [116]:
df = df.join(df["Extras"].str.get_dummies(sep=",").add_prefix("ex_"))

# Safety & Security

In [117]:
liststring(df, "Safety & Security")

In [118]:
emptytonan(df, "Safety & Security")

In [119]:
fill_most(df, "make_model", "Safety & Security")

In [120]:
df = df.join(df["Safety & Security"].str.get_dummies(sep=",").add_prefix("ss_"))

# Get_Dummies

In [121]:
cat_list = ["Make", "Paint Type", "make_model", "body_type", "vat", "Type", "Model", "Body Color", "Gearing Type", "Drive chain", "Fuel", "Emission Class", "Country version"]

In [122]:
df = df.join(df["Make"].str.get_dummies(sep=",").add_prefix("mk_"))

In [123]:
df = df.join(df["Paint Type"].str.get_dummies(sep=",").add_prefix("pt_"))

In [124]:
df = df.join(df["make_model"].str.get_dummies(sep=",").add_prefix("mm_"))

In [125]:
df = df.join(df["body_type"].str.get_dummies(sep=",").add_prefix("bt_"))

In [126]:
df = df.join(df["vat"].str.get_dummies(sep=",").add_prefix("vat_"))

In [127]:
df = df.join(df["Type"].str.get_dummies(sep=",").add_prefix("type_"))

In [128]:
df = df.join(df["Model"].str.get_dummies(sep=",").add_prefix("model_"))

In [129]:
df = df.join(df["Body Color"].str.get_dummies(sep=",").add_prefix("bc_"))

In [130]:
df = df.join(df["Gearing Type"].str.get_dummies(sep=",").add_prefix("gt_"))

In [131]:
df = df.join(df["Drive chain"].str.get_dummies(sep=",").add_prefix("dc_"))

In [132]:
df = df.join(df["Fuel"].str.get_dummies(sep=",").add_prefix("fl_"))

In [133]:
df = df.join(df["Emission Class"].str.get_dummies(sep=",").add_prefix("ec_"))

In [134]:
df = df.join(df["Country version"].str.get_dummies(sep=",").add_prefix("cv_"))

In [135]:
df = df.drop(["Make", "Paint Type", "body_type", "vat", "Type", "Model", "Body Color", "Gearing Type", "Drive chain", "Fuel", "Emission Class", "Country version", "Comfort & Convenience", "Entertainment & Media", "Extras", "Safety & Security"], axis=1)

# Upholstery

In [136]:
df["Upsurf"] = df["Upholstery"].str.split(",").str[0][df["Upholstery"].str.split(",").str[0].str.contains("Cloth|Part leather|Full leather|Velour|alcantara")]

In [137]:
fill_most(df, "make_model", "Upsurf")

In [138]:
df["Upcolor"] = df["Upholstery"].str.split(",").str[-1][df["Upholstery"].str.split(",").str[-1].str.contains(" Black| Grey| Brown|Black|Grey| Beige| Blue|Brown| White| Red| Yellow|Blue|White| Orange")]

In [139]:
df["Upcolor"] = df["Upcolor"].str.strip()

In [140]:
fill_most(df, "Upsurf", "Upcolor")

In [141]:
df = pd.get_dummies(data=df, columns=["Upsurf", "Upcolor"])

In [142]:
df.drop(["make_model", "Upholstery"], axis=1, inplace=True)

# Warranty

In [143]:
df.Warranty = df.Warranty.str.extract("(\d\d|\d)")[0]

In [144]:
df.Warranty = df.Warranty.astype("int")

# registration

In [145]:
df.registration = pd.to_datetime(df.registration, format="%m/%Y")

# Gears

In [146]:
df.Gears = df.Gears.astype("int")

In [147]:
df.Gears[df.Gears>8] = 6

In [148]:
Gearswin = winsorize(df.Gears, (0.01, 0.01))

In [149]:
df["Gears"] = pd.DataFrame(Gearswin)[0]

# CO2 Emission

In [150]:
df["CO2 Emission"] = df["CO2 Emission"].astype("int")

In [151]:
df["CO2 Emission"].describe()

count    15919.000000
mean       121.500471
std        248.536459
min          0.000000
25%        102.000000
50%        116.000000
75%        127.000000
max      14457.000000
Name: CO2 Emission, dtype: float64

In [152]:
co2 = winsorize(df["CO2 Emission"], (0.01, 0.02))

In [153]:
df["CO2 Emission"] = pd.DataFrame(co2)[0]

# Consumption

In [154]:
df["Consumption"] = df["Consumption"].astype("float")

In [155]:
consump = winsorize(df["Consumption"], (0.01, 0.01))

In [156]:
df["Consumption"] = pd.DataFrame(consump)[0]

# Cylinders

In [157]:
df["Cylinders"] = df["Cylinders"].astype("int")

In [158]:
win = winsorize(df["Cylinders"], (0.01, 0.01))

In [159]:
df["Cylinders"] = pd.DataFrame(win)[0]

# Displacement

In [160]:
df["Displacement"] = df["Displacement"].astype("int")

In [161]:
win = winsorize(df["Displacement"], (0.01, 0.01))

In [162]:
df["Displacement"] = pd.DataFrame(win)[0]

# Nr. of Seats

In [163]:
df["Nr. of Seats"] = df["Nr. of Seats"].astype("int")

In [164]:
win = winsorize(df["Nr. of Seats"], (0.01, 0.01))

In [165]:
df["Nr. of Seats"] = pd.DataFrame(win)[0]

# Nr. of Doors

In [166]:
df["Nr. of Doors"] = df["Nr. of Doors"].astype("int")

In [167]:
win = winsorize(df["Nr. of Doors"], (0.01, 0.01))

In [168]:
df["Nr. of Doors"] = pd.DataFrame(win)[0]

# Previous Owners

In [169]:
df["Previous Owners"].value_counts(dropna=False)

1    13520
0     1602
2      778
3       17
4        2
Name: Previous Owners, dtype: int64

In [170]:
df["Previous Owners"] = df["Previous Owners"].astype("int")

# hp

In [171]:
df["hp"] = df["hp"].astype("int")

In [172]:
win = winsorize(df["hp"], (0.01, 0.03))

In [173]:
df["hp"] = pd.DataFrame(win)[0]

# km

In [174]:
df["km"] = df["km"].astype("int")

In [175]:
win = winsorize(df["km"], (0.01, 0.02))

In [176]:
df["km"] = pd.DataFrame(win)[0]

# Price

In [177]:
df["price"] = df["price"].astype("int")

In [178]:
win = winsorize(df["price"], (0.01, 0.02))

In [179]:
df["price"] = pd.DataFrame(win)[0]

# Tipleri Ayarlama

In [180]:
for i in df.iloc[:, 200:214].columns:
    df[i] = df[i].astype("int64")

In [181]:
for i in ["price", "km", "hp", "Previous Owners", "Warranty", "Nr. of Doors", "Nr. of Seats", "Displacement", "Cylinders", "CO2 Emission", "Gears"]:
    df[i] = df[i].astype("int64")

# Datetimelar

In [182]:
df["Next Inspection"][df["Next Inspection"]<"2016-01-01"].index

Int64Index([1169, 3308, 4250, 6674, 7155, 7837, 7893, 13068, 13332], dtype='int64')

In [183]:
for i in [1169, 3308, 4250, 6674, 7155, 7837, 7893, 13068, 13332]:
    df["Next Inspection"][i] = df["Next Inspection"][13234]

In [184]:
df["age_year"] = df["registration"].apply(lambda x: 2019-int(x.strftime('%Y')))

In [185]:
df["age_year"][df["age_year"]<0] = 0

In [186]:
df["age_year"].value_counts()

1    4525
0    4444
3    3675
2    3275
Name: age_year, dtype: int64

In [187]:
df["yeartoinspection"] = df["Next Inspection"].apply(lambda x: int(x.strftime('%Y'))-2019)

In [188]:
df["yeartoinspection"][df["yeartoinspection"]<0] = 0

In [189]:
df["yeartoinspection"].value_counts()

2    6761
3    4280
1    3291
0    1539
4      47
5       1
Name: yeartoinspection, dtype: int64

In [190]:
df.drop(["registration", "Next Inspection"], axis=1, inplace=True)

# SONUÇ

In [195]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [192]:
df.isnull().sum()

price               0
km                  0
hp                  0
Previous Owners     0
Warranty            0
                   ..
Upcolor_Red         0
Upcolor_White       0
Upcolor_Yellow      0
age_year            0
yeartoinspection    0
Length: 214, dtype: int64

In [193]:
df.describe()

,price,km,hp,Previous Owners,Warranty,Nr. of Doors,Nr. of Seats,Displacement,Cylinders,Consumption,CO2 Emission,Gears,cc_Air conditioning,cc_Air suspension,cc_Armrest,cc_Automatic climate control,cc_Auxiliary heating,cc_Cruise control,cc_Electric Starter,cc_Electric tailgate,cc_Electrical side mirrors,cc_Electrically adjustable seats,cc_Electrically heated windshield,cc_Heads-up display,cc_Heated steering wheel,cc_Hill Holder,cc_Keyless central door lock,cc_Leather seats,cc_Leather steering wheel,cc_Light sensor,cc_Lumbar support,cc_Massage seats,cc_Multi-function steering wheel,cc_Navigation system,cc_Panorama roof,cc_Park Distance Control,cc_Parking assist system camera,cc_Parking assist system self-steering,cc_Parking assist system sensors front,cc_Parking assist system sensors rear,cc_Power windows,cc_Rain sensor,cc_Seat heating,cc_Seat ventilation,cc_Split rear seats,cc_Start-stop system,cc_Sunroof,cc_Tinted windows,cc_Wind deflector,cc_Windshield,...,dc_front,dc_rear,fl_Benzine,fl_Diesel,fl_Electric,fl_Gasoline,ec_Euro 4,ec_Euro 5,ec_Euro 6,ec_Euro 6c,ec_Euro 6d-TEMP,cv_Austria,cv_Belgium,cv_Bulgaria,cv_Croatia,cv_Czech Republic,cv_Denmark,cv_Egypt,cv_European Union,cv_France,cv_Germany,cv_Hungary,cv_Italy,cv_Japan,cv_Luxembourg,cv_Netherlands,cv_Poland,cv_Romania,cv_Serbia,cv_Slovakia,cv_Slovenia,cv_Spain,cv_Sweden,cv_Switzerland,Upsurf_Cloth,Upsurf_Full leather,Upsurf_Part leather,Upsurf_Velour,Upsurf_alcantara,Upcolor_Beige,Upcolor_Black,Upcolor_Blue,Upcolor_Brown,Upcolor_Grey,Upcolor_Orange,Upcolor_Red,Upcolor_White,Upcolor_Yellow,age_year,yeartoinspection
count,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.00000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,...,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000,15919.000000
mean,17915.871474,31442.665494,87.840756,0.950751,15.026384,4.660594,4.967586,1426.501539,3.813619,4.822709,116.320309,5.810666,0.949180,0.003895,0.476035,0.577989,0.016710,0.730448,0.000063,0.075633,0.797789,0.129845,0.058860,0.038068,0.202148,0.462152,0.181167,0.003015,0.611785,0.514668,0.224826,0.022614,0.735285,0.535147,0.039010,0.679188,0.231924,0.048244,0.388906,0.641686,0.931842,0.563352,0.466801,0.05239,0.149947,0.590615,0.023368,0.090709,0.002576,0.000754,...,0.986997,0.000251,0.332496,0.458886,0.000314,0.208305,0.002513,0.004900,0.858722,0.007978,0.125887,0.013066,0.019725,0.000126,0.000251,0.003267,0.002073,0.000063,0.031849,0.002387,0.804699,0.001759,0.065205,0.002073,0.000063,0.029148,0.003078,0.000126,0.000063,0.000251,0.000063,0.020416,0.000188,0.000063,0.810101,0.088385,0.094164,0.003769,0.003581,0.003204,0.873170,0.001005,0.033482,0.087443,0.000063,0.000565,0.000817,0.000251,1.388278,1.874867
std,6961.480413,35321.259895,24.611395,0.390504,9.833687,0.641626,0.409959,275.310364,0.389426,0.849664,16.711693,0.691128,0.219636,0.062288,0.499441,0.493896,0.128185,0.44374

In [ ]:
df

In [295]:
df.to_csv("autoproject.csv", index=False)